# Product Matching

Matching a query product image against a set of reference product images and finds the reference products that are most similar to the query product.

In [6]:
import sys
sys.path.append("../python_code")
from AttrPredModel import AttrPred_Resnet50
from iMatDataset import iMatDataset
import matplotlib.pyplot as plt
import torch

### loading reference dataset

In [4]:
ds = iMatDataset(labels_file="../Data/iMaterialist/validation.json", img_dir="../Data/iMaterialist/validation", attr_descr_file="../Data/iMaterialist/iMat_fashion_2018_label_map_228.csv",
                rgb_mean=[0.6742, 0.6307, 0.6173], rgb_std=[0.6742, 0.6307, 0.6173])

### loading model

In [5]:
# path to downloaded model file. Must be adapted to fit your file structure!
model_path = "../TrainedModels/epoch=3.ckpt"
model = AttrPred_Resnet50.load_from_checkpoint(model_path)
model.eval()
print("Model Loaded successfully")

Model Loaded successfully


### analyzing reference product images

In [9]:
ref_attr_scores = [model(image_tensor.unsqueeze(0))[0] for image_tensor,_ in ds]

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 38535168 bytes. Buy new RAM!
